In [1]:
import pandas
import json 
import requests
import datetime
import time
import openpyxl as xl

In [28]:
# 입력부

# ▼ 로그인 할 아이디와 비밀번호 입력 ▼
credential = {
    "email": "아이디",
    "password": "비밀번호"
}


# ▼ 겟사콘 엑셀 경로 입력 ▼
# 예시 ) filename="파일파일.xlsx"
# 예시 ) filename="C:\각종문서\python_work_space\파일파일.xlsx"
# filename = '예시와 같이 파일경로 입력'
# ipynb 파일과 같은 폴더 안에 있는 경우 파일명만 입력하면 됨

filename = "샘플.xlsx"


In [ ]:
# 1.어드민 엑세스 토큰 받는곳

URL = 'https://identityaccess.api.modusign.co.kr/v1/admin/users/'
TOKEN_URL = 'https://identityaccess.api.modusign.co.kr/v1/user/oauth/token'
X_API_KEY = 'FidrahieciWU9wQrieCY4rjODW5WGIbaeFr8Li8e'
data = json.dumps(credential)
headers = {
    "x-api-key": X_API_KEY,
    "contentType": "application/json",
    "authorization": "none"
}
res = requests.post(TOKEN_URL, data = data, headers = headers)
res_json = res.json()
TOKEN = res_json['result']['access_token']


In [29]:
# 2.userId 와 사업자 번호 엑셀 파일 불러오는 부분
data = pandas.read_excel(filename)
df_sample = data[["userId",'"사업자등록번호"를 입력해주세요.']]
df_sample.rename(columns = {"userId" : 'userid','"사업자등록번호"를 입력해주세요.' : 'business_registration_number'},inplace = True)
userlist = []
temp = []
number = 0
for index, i in df_sample.iterrows() :
    userlist.append(number)
    temp.append(dict(i))
    number = number + 1
userId = dict(zip(userlist,temp))

In [ ]:
# 3.매크로 실행부

AUTHORIZATION = "Bearer " + TOKEN
final_result = []
failed_count = 0
process_count = 0

# 진행상황
def alert() : 
    global process_count
    process_count += 1
    percent = process_count/len(userId)*100
    percent = round(percent, 2)
    print(str(percent) + "% 진행 중")
    


# function 유저 설문 정보 바꾸기
def change_user_data(value1) :
    # function 유저 설문 정보 가져오기
    headers = {
        "x-api-key": X_API_KEY,
        "contentType": 'application/json',
        "authorization": AUTHORIZATION
    }
    URL_get = URL + str(value1['userid'])
    temp1 = requests.get(URL_get,headers=headers)
    jsonObjs = temp1.json()
    temp2 = jsonObjs['survey']
 
    

    # $$▼ 실행 결과 저장용 ▼$$
    result = []
    result.append(userId[i]['userid'])
    result.append(jsonObjs['user_id'])
    result.append(jsonObjs['email'])
    result.append(jsonObjs['survey']['business_registration_number'])
    # $$▲ 실행 결과 저장용 ▲$$

    # function 유저 설문 정보 갱신
    URL_put = URL + str(value1['userid']) + "/survey"
    number_value = str(value1['business_registration_number']).replace("-","").replace(" ","")
    temp2.update({'business_registration_number' : number_value})
    output_value = requests.put(URL_put, data=temp2, headers=headers)
    alert()
    
    if output_value.status_code == 200:
        status_code_str = "성공"
        result.append(status_code_str)
        result.append(res.status_code)   
        result.append(number_value)
    else :
        global failed_count
        failed_count += 1
        status_code_str = "오류"
        result.append(status_code_str)
        result.append(res.status_code)   
        result.append(number_value)
    
    time.sleep(0.5)
    return final_result.append(result)

start_time = datetime.datetime.now()

# body
for i in userId :
    change_user_data(userId[i])
    

finish_time = datetime.datetime.now()   
finish_time_value = '{:%Y%m%d%H%M%S}'.format(finish_time)
print("========================================\n - 시작 시각은 " + start_time.strftime("%Y-%m-%d %H:%M:%S") + "이며, \n - 종료된 시각은 " + finish_time.strftime("%Y-%m-%d %H:%M:%S") + "입니다.\n")
print(' - 사업자번호 등록이 실패된 유저수는 ' + str(failed_count) + ' 명 입니다.')
columns = ["검색한 유저","검색된 유저","이메일","변경전 사업자 번호", "성공여부","응답코드", "변경한 사업자 번호"]
df = pandas.DataFrame(final_result, columns = columns)
df.to_excel('result({0}).xlsx'.format(finish_time_value),sheet_name='result')
df


In [ ]:
# 오류시 중간저장용

finish_time = datetime.datetime.now()   
finish_time_value = '{:%Y%m%d%H%M%S}'.format(finish_time)
print("========================================\n - 시작 시각은 " + start_time.strftime("%Y-%m-%d %H:%M:%S") + "이며, \n - 종료된 시각은 " + finish_time.strftime("%Y-%m-%d %H:%M:%S") + "입니다.\n")
print(' - 사업자번호 등록이 실패된 유저수는 ' + str(failed_count) + ' 명 입니다.')
columns = ["검색한 유저","검색된 유저","이메일","변경전 사업자 번호", "성공여부","응답코드", "변경한 사업자 번호"]
df = pandas.DataFrame(final_result, columns = columns)
df.to_excel('result({0}).xlsx'.format(finish_time_value),sheet_name='result')
df